# MoodMate: A Sentiment-Responsive Question-Answering Psychologist Chatbot for Emotional Support By Fine-tuning Llama-3.2-3B-Instruct Model

Developed by **Group 9**:

* Jiesen LONG, 2230026109
* Zhiyue CHOU, 2230026028
* Jiaying SHAN, 2230026230
* Yizhen XIN, 2230026179
* Ruoyi WANG, 2230026155
* Shuxuan DING, 2230026232


In [1]:
import random
import gc
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import numpy as np
import pandas as pd
import transformers
import accelerate
import bitsandbytes as bnb
from datasets import load_dataset, concatenate_datasets
import torch

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

In [2]:
# Load the training dataset
dataset = load_dataset("json", data_files="dataset/combined_data.json", split="train")

In [3]:
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 50328
})

In [4]:
dataset['input'][0]

'心理咨询师，我觉得我很自私，因为我总是关心自己的感受，有时候会忽略别人的感受。'

In [5]:
dataset['output'][0]

'你能详细描述一下这种情况吗？例如在什么情况下你会觉得自己自私？'

In [6]:
dataset['input'][50000]

"My daughter is in later elementary school. She can't color in the lines. Her words jumble together when she writes unless there are big spaces or she skips lines."

In [7]:
dataset['output'][50000]

'There could be a number of things going on here. For instance, have her eyes been checked by an optometrist? She might just not like writing or coloring. She could be rushing through assignments so that she can spend time with friends, play games, or do something else. She might need some extra help with fine motor skills. What are her grades like? Does she rush through other things like cleaning her room or getting ready for bed? '

In [8]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

#Load Tokenizer
tokenizer= AutoTokenizer.from_pretrained('model_3b')

# Add Padding Token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the LLaMA model in 4-bit
model = transformers.AutoModelForCausalLM.from_pretrained(
    "model_3b",
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

In [10]:
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [11]:
args = TrainingArguments(
    output_dir="CounselLlama1B",
    logging_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_checkpointing=False,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    load_best_model_at_end=True,
    eval_strategy='epoch'
)

In [12]:
# System message to better instruct chatbot
system_message = """You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Remember, your goal is to provide a supportive and understanding environment for users to share their feelings and concerns. Always prioritize their well-being and safety."""

def format_llama(entry):
    formatted = f"<s>[INST] <<SYS>>{system_message}<</SYS>>{entry['input']} [/INST]  {entry['output']}  </s>"

    return formatted

In [13]:
# Split the dataset in 8:2
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset_split['train']
valid_dataset = dataset_split['test']

In [14]:
print(format_llama(train_dataset[0]))

<s>[INST] <<SYS>>You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Reme

In [15]:
max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_llama,
    args=args,
)

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/miniconda3/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Us

In [16]:
gc.collect()
torch.cuda.empty_cache()

In [17]:
trainer.args.gradient_checkpointing = False

# train
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,0.492000,0.460894
2,0.603800,0.440682
3,0.394500,0.428281
4,0.514800,0.419215
5,0.408500,0.412057
6,0.904500,0.404542
7,0.387600,0.399189
8,0.366300,0.394552
9,0.378000,0.390680
10,0.336400,0.387674


TrainOutput(global_step=14900, training_loss=0.4174032885916281, metrics={'train_runtime': 41788.6875, 'train_samples_per_second': 1.426, 'train_steps_per_second': 0.357, 'total_flos': 2.0767444550497075e+18, 'train_loss': 0.4174032885916281, 'epoch': 10.0})

In [18]:
# Save model
trainer.save_model("./fine_tuned_model_3b")